Если хотим использовать классификатор, который не возвращает вероятности принадлежности меток к классам, то придётся править код в sentiment_classifier.py (я поигрался, удалил всё что связано с predict_proba, но всё же решил использовать LogReg вместо LinearSVC)

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
import numpy as np
import nltk
from nltk.corpus import movie_reviews
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
import joblib

In [2]:
nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

In [3]:
#Чтобы получить id-шники негативных и позитивных отзывов:
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')
#Чтобы получить список негативных отзывов:
negfeats = [movie_reviews.words(fileids=[f]) for f in negids]
#Чтобы получить список позитивных отзывов:
posfeats = [movie_reviews.words(fileids=[f]) for f in posids]

In [4]:
corpus = negfeats + posfeats
y0 = np.zeros(1000)
y1 = np.ones(1000)
y = np.hstack((y0, y1))

In [5]:
corpus1 = [" ".join(x) for x in corpus]

In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [7]:
stop_nltk = nltk.corpus.stopwords.words('english')

In [12]:
vectorizer = CountVectorizer(ngram_range=(1,2), stop_words=stop_nltk, min_df=10)

In [13]:
vectorizer.fit(corpus1)

CountVectorizer(min_df=10, ngram_range=(1, 2),
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...])

In [14]:
joblib.dump(vectorizer, 'BigramUnprocessedVectorizer.pkl')

['BigramUnprocessedVectorizer.pkl']

In [23]:
#pipe = Pipeline([('vect', CountVectorizer(ngram_range=(1,2), stop_words=stop_nltk, min_df=10)),
#                  ('clf', LogisticRegression())])

In [24]:
#estimator = pipe
#cv_results = cross_val_score(estimator, corpus1, y, cv=5, scoring='accuracy')
#print('cv_results.mean: ',cv_results.mean())

In [26]:
classifer = LogisticRegression()
X = vectorizer.fit_transform(corpus1)
classifer.fit(X, y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [28]:
joblib.dump(classifer, 'DefaultLogisticBigramUnprocessedTextSentiment.pkl')

['DefaultLogisticBigramUnprocessedTextSentiment.pkl']

In [25]:
#pipe.clf.best_params_